In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor  

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import mlflow
import mlflow.sklearn
# Load the data
train_store_df = pd.read_csv(r"C:\Users\VARUN\Downloads\merged.csv")
train_store_df.info()
# convert date in datetimeindex format

train_store_df['Date'] = pd.DatetimeIndex(train_store_df['Date'])
train_store_df['Year'] = pd.DatetimeIndex(train_store_df['Date']).year
train_store_df['Month'] = pd.DatetimeIndex(train_store_df['Date']).month
train_store_df['Day'] = pd.DatetimeIndex(train_store_df['Date']).day
# train_store_df['WeekOfYear'] = train_store_df['Date'].dt.weekofyear
train_store_df['WeekOfYear'] = train_store_df['Date'].dt.isocalendar().week


# columns to be use:['DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday','SchoolHoliday', 'Year', 'Month', 'Day', 'WeekOfYear']

# Lable Date and StateHoiday Train:

columns = ['Sales','Store', 'DayOfWeek', 'Open', 'Promo',  'SchoolHoliday', 'Day', 'WeekOfYear','Month', 'Year', 'StoreType',
              'Assortment','CompetitionDistance', 'Promo2']

feature_columns = ['Store', 'DayOfWeek', 'Open', 'Promo',  'SchoolHoliday', 'Day', 'WeekOfYear','Month', 'Year', 'StoreType',
              'Assortment','CompetitionDistance', 'Promo2']

sample_size = 1000
sampled_df = train_store_df[columns].sample(sample_size)


# Separate Test and train files:
train_x = sampled_df[feature_columns]
train_y = sampled_df[['Sales']]
train_x.head()

# Label encoding: 

lb = LabelEncoder()
train_x['StoreType'] = lb.fit_transform(train_x['StoreType'])
train_x['Assortment'] = lb.fit_transform(train_x['Assortment'])

train_x.info()
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

# X = x_scaler.fit_transform(train_x)
X = train_x
Y = y_scaler.fit_transform(train_y)
#split the data inti train test 

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=42)

#create the model

rf_reg = RandomForestRegressor(n_estimators = 100, max_depth=25, random_state=0)
#fit the model

rf_reg.fit(X_train, y_train)

#Make the prediction

score = rf_reg.score(X_test, y_test)

print(f"Prediction Score of the Model is {round(score * 100, 2)}%")

# ... (previous imports and data preprocessing)

def train_model(X, Y, model_type='LinearRegression'):
    model = LinearRegression()
    
    if model_type == 'RandomForest':
        model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=0)
    elif model_type == 'DecisionTree':
        model = DecisionTreeRegressor(random_state=0)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    for train_index, test_index in kf.split(X):
        with mlflow.start_run(run_name=model_type):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y[train_index], Y[test_index]

            model.fit(X_train, y_train)
            score = model.score(X_test, y_test)
            
            # Evaluate the model on additional metrics
            y_pred = model.predict(X_test)
            mae = mean_absolute_error(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)

            print(f"Prediction Score of the {model_type} is {round(score * 100, 2)}%")
            print(f"Mean Absolute Error: {mae}, Mean Squared Error: {mse}")

            if model_type == 'LinearRegression':
                mlflow.log_param('LR-Score', score)
            elif model_type == 'RandomForest':
                mlflow.log_param('RF-Score', score)
            elif model_type == 'DecisionTree':
                mlflow.log_param('DT-Score', score)

            mlflow.sklearn.log_model(model, model_type + ' Model')
            mlflow.end_run()
            
    print()

# Run the training for all models
train_model(X, Y, "RandomForest")
train_model(X, Y, "DecisionTree")
train_model(X, Y, "LinearRegression")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 22 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   index                      1017209 non-null  int64  
 1   Store                      1017209 non-null  int64  
 2   DayOfWeek                  1017209 non-null  int64  
 3   Date                       1017209 non-null  object 
 4   Sales                      1017209 non-null  float64
 5   Customers                  1017209 non-null  float64
 6   Open                       1017209 non-null  int64  
 7   Promo                      1017209 non-null  int64  
 8   StateHoliday               1017209 non-null  int64  
 9   SchoolHoliday              1017209 non-null  float64
 10  StoreType                  1017209 non-null  int64  
 11  Assortment                 1017209 non-null  int64  
 12  CompetitionDistance        1017209 non-null  float64
 13  CompetitionO

2024/02/02 11:17:00 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



Prediction Score of the RandomForest is 66.0%
Mean Absolute Error: 0.09678122606419848, Mean Squared Error: 0.018869270325591575
Prediction Score of the RandomForest is 58.15%
Mean Absolute Error: 0.11214698723395652, Mean Squared Error: 0.023780564701548358
Prediction Score of the RandomForest is 60.18%
Mean Absolute Error: 0.10737648068895296, Mean Squared Error: 0.021111845137232875
Prediction Score of the RandomForest is 63.7%
Mean Absolute Error: 0.10293853727558652, Mean Squared Error: 0.019892638112813152
Prediction Score of the RandomForest is 66.36%
Mean Absolute Error: 0.10077694529467292, Mean Squared Error: 0.018882512855109165

Prediction Score of the DecisionTree is 26.93%
Mean Absolute Error: 0.13999460586881474, Mean Squared Error: 0.040554413877678074
Prediction Score of the DecisionTree is 26.94%
Mean Absolute Error: 0.1471666426927503, Mean Squared Error: 0.04151694372230723
Prediction Score of the DecisionTree is 23.48%
Mean Absolute Error: 0.14484141254315305, Mean